In [1]:
import sys
import time
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix, issparse
from scipy.stats import iqr
from bionemo.scdl.io.single_cell_memmap_dataset import SingleCellMemMapDataset
from torch.utils.data import DataLoader
from functools import wraps
import math
from enum import Enum

import subprocess
from bionemo.scdl.util.torch_dataloader_utils import collate_sparse_matrix_batch


In [2]:
help(SingleCellMemMapDataset)

Help on class SingleCellMemMapDataset in module bionemo.scdl.io.single_cell_memmap_dataset:

class SingleCellMemMapDataset(bionemo.scdl.api.single_cell_row_dataset.SingleCellRowDataset)
 |  SingleCellMemMapDataset(data_path: str, h5ad_path: Optional[str] = None, num_elements: Optional[int] = None, num_rows: Optional[int] = None, mode: bionemo.scdl.io.single_cell_memmap_dataset.Mode = 'r+', dtypes: Dict[bionemo.scdl.io.single_cell_memmap_dataset.FileNames, str] = {'data.npy': 'float32', 'col_ptr.npy': 'uint32', 'row_ptr.npy': 'uint64'}) -> None
 |  
 |  Represents one or more AnnData matrices.
 |  
 |  Data is stored in large, memory-mapped arrays that enables fast access of
 |  datasets larger than the available amount of RAM on a system. SCMMAP
 |  implements a consistent API defined in SingleCellRowDataset.
 |  
 |  Attributes:
 |      data_path: Location of np.memmap files to be loaded from or that will be
 |      created.
 |      mode: Whether the dataset will be read in (r+) from 

In [3]:
class FileNames(str, Enum):
    """Names of files that are generated in SingleCellCollection."""

    DATA = "data.npy"
    COLPTR = "col_ptr.npy"
    ROWPTR = "row_ptr.npy"
    METADATA = "metadata.json"
    DTYPE = "dtypes.json"
    FEATURES = "features"
    VERSION = "version.json"



In [4]:
def get_disk_size(directory):
    result = subprocess.run(['du', '-sb', directory], stdout=subprocess.PIPE, text=True)
    size_in_bytes = int(result.stdout.split()[0])
    return size_in_bytes
def timeit(method):
    @wraps(method)
    def timed(*args, **kwargs):
        start_time = time.time()
        result = method(*args, **kwargs)
        end_time = time.time()
        run_time = end_time - start_time
        print(f"Method {method.__name__} took {run_time:.4f} seconds")
        return result, run_time
    return timed

def time_all_methods(cls):
    for attr_name, attr_value in cls.__dict__.items():
        if callable(attr_value) and attr_name != "__init__":  # Check if the attribute is a method
            setattr(cls, attr_name, timeit(attr_value))
    return cls


In [5]:
@time_all_methods
class AnnDataMetrics:
    def __init__(self, adatapath):
        self.adatapath = adatapath
    def load_backed(self):
        self.adata_backed = ad.read_h5ad(self.adatapath, backed=True)

    def load_whole(self):
        self.adata = ad.read_h5ad(self.adatapath, backed=False)

    def max(self):
        return self.adata.X.data.max()

    def min(self):
        return self.adata.X.data.min()

    def mean(self):
        return self.adata.X.data.mean()

    def median(self):
        return np.median(self.adata.X.data)

    def interQuartRange(self):
        return iqr(self.adata.X.data)

    def num_values(self):
        return self.adata.X.shape[0] * self.adata.X.shape[1]
    
    def data_type(self):
        return self.adata.X.data.dtype
    
    def get_int_size(self, val): 
        if val <= 8:
            return "uint8"
        elif val <= 16:
            return "uint16"
        elif val <= 32:
            return "uint32"
        else:
            return "uint64"

    def row_width(self):
        return self.get_int_size(math.log2(self.adata.X.indptr[-1]))
    
    def column_width(self):
        return self.get_int_size(math.log2(self.adata.X.indices[-1]))
    
    def sparsity_stats(self):
        num_non_zero = 0
        # Get the number of non-zero values
        if issparse(self.adata.X):
            num_non_zero = self.adata.X.nnz
        else:
            num_non_zero = (self.adata.X.round() != 0).sum()
            
        num_vals = self.num_values(self.adata)
        num_zeros = num_vals - num_non_zero
        sparsity = num_zeros / num_vals
        
        return num_zeros, num_non_zero, sparsity

    def size_disk_bytes(self):
        return get_disk_size(self.adatapath)
    def size_adata_bytes(self): 
        return sys.getsizeof(self.adata)
    def size_adata_backed_bytes(self): 
        return sys.getsizeof(self.adata_backed)

    def random_rows_whole(self, random_samples = 10_000):
        L = self.adata.X.shape[0]     
        rIdx = np.sort(np.random.choice(L, size=(random_samples,), replace=True))
        x = self.adata[rIdx, :].X
        return x
    def random_rows_backed(self, random_samples = 10_000):
        L = self.adata_backed.X.shape[0]     
        rIdx = np.sort(np.random.choice(L, size=(random_samples,), replace=True))
        x = self.adata[rIdx, :].X
        return x
    def random_values_whole(self, random_samples = 10_000):
        rows = self.adata.X.shape[0]
        cols = self.adata.X.shape[1]
        rIdx = np.sort(np.random.choice(rows, size=(random_samples), replace=True))
        cIdx = np.sort(np.random.choice(cols, size=(random_samples), replace=True))
        return [self.adata.X[r,c] for r,c in zip(rIdx, cIdx)]
    
    def random_values_backed(self, random_samples = 10_000):
        rows = self.adata.X.shape[0]
        cols = self.adata.X.shape[1]
        rIdx = np.sort(np.random.choice(rows, size=(random_samples), replace=True))
        cIdx = np.sort(np.random.choice(cols, size=(random_samples), replace=True))
        return [self.adata_backed.X[r,c] for r,c in zip(rIdx, cIdx)]


In [6]:

@time_all_methods
class SCDLMetrics:
    def __init__(self, adatapath, memmap_dir, row_width, column_width, data_type):
        self.adatapath = adatapath
        self.memmap_dir = memmap_dir
        self.row_width = row_width
        self.column_width = column_width
        self.data_type = data_type
        
    def create_from_adata(self):
        self.first_ds = SingleCellMemMapDataset(self.memmap_dir, 
                                                self.adatapath,
                                                        )

    def save(self):
        self.first_ds.save()
    def load_backed(self):
        self.ds = SingleCellMemMapDataset(self.memmap_dir)
    def max(self):
        return np.max(self.ds.data)

    def min(self):
        return np.min(self.ds.data)

    def mean(self):
        return np.mean(self.ds.data)

    def median(self):
        return np.median(self.ds.data)

    def interQuartRange(self):
        return iqr(self.ds.data)

    def num_values(self):
        return self.ds.number_of_values()

    def sparsity_stats(self):
        return self.ds.sparsity()
    
    
    def size_disk_bytes(self):
        return get_disk_size(self.memmap_dir)
    def size_mem_dataset_bytes(self): 
        return sys.getsizeof(self.ds)

    def random_rows(self, random_samples = 10_000):
        L = self.ds.number_of_rows()   
        rIdx = np.sort(np.random.choice(L, size=(random_samples), replace=True))
        return [self.ds[v] for v in rIdx]
    
    def random_values(self, random_samples = 10_000):
        rows = self.ds.number_of_rows() 
        cols = self.ds.shape()[1][0]  
        rIdx = np.sort(np.random.choice(rows, size=(random_samples), replace=True))
        cIdx = np.sort(np.random.choice(cols, size=(random_samples), replace=True))
        return [self.ds.get_row_column(r,c) for r,c in zip(rIdx, cIdx)]

    def iterate_dl(self, batch_size = 8):
        model = lambda x : x

        dataloader = DataLoader(self.ds, batch_size=batch_size, shuffle=True, collate_fn=collate_sparse_matrix_batch)
        n_epochs = 1
        for e in range(n_epochs):
            for batch in dataloader:
                model(batch)


In [7]:
results_dict = {}

In [8]:
fn = "97e96fb1-8caf-4f08-9174-27308eabd4ea.h5ad"
anndatapath = "examples/hdf5s/" + fn
results_dict["anndata file"] = fn 
anndata_m = AnnDataMetrics(anndatapath)
results_dict["AnnData Dataset Backed Load Time (s)"] = anndata_m.load_backed()[1]
results_dict["AnnData Dataset Load Time (s)"] = anndata_m.load_whole()[1]

results_dict["AnnData Dataset Max Calculation Time (s)"] = anndata_m.max()[1]
results_dict["AnnData Dataset Min Calculation Time (s)"] = anndata_m.min()[1]
results_dict["AnnData Dataset Mean Calculation Time (s)"] = anndata_m.mean()[1]
results_dict["AnnData Dataset Size on Disk (MB)"] = anndata_m.size_disk_bytes()[0]/(1_024**2)

results_dict["AnnData Dataset Size on Disk (MB)"] = anndata_m.size_disk_bytes()[0]/(1_024**2)
results_dict["AnnData Dataset Size in Memory (MB)"] = anndata_m.size_adata_bytes()[0]/(1_024**2)
results_dict["AnnData Backed Dataset Size in Memory (MB)"] = anndata_m.size_adata_backed_bytes()[0]/(1_024**2)


Method load_backed took 0.2145 seconds
Method load_whole took 1.4693 seconds
Method max took 0.0050 seconds
Method min took 0.0049 seconds
Method mean took 0.0057 seconds
Method size_disk_bytes took 0.0016 seconds
Method size_disk_bytes took 0.0018 seconds
Method size_adata_bytes took 0.0155 seconds
Method size_adata_backed_bytes took 0.0150 seconds


In [9]:
results_dict["AnnData Time to retrieve a random batch 100 values loaded in memory (s)"] = anndata_m.random_values_whole(random_samples = 100)[1]
results_dict["AnnData Time to retrieve a random batch 100 values backed on disk (s)"] = anndata_m.random_values_backed(random_samples = 100)[1]
results_dict["AnnData Time to retrieve a random batch of 100 cells loaded in memory (s)"] = anndata_m.random_rows_whole(random_samples = 100)[1]
results_dict["AnnData Time to retrieve a random batch of 100 cells backed on disk (s)"] = anndata_m.random_rows_backed(random_samples = 100)[1]

Method random_rows_whole took 0.0140 seconds
Method random_rows_backed took 0.0122 seconds


In [10]:
data_type = str(anndata_m.data_type()[0])
results_dict["data type"] = data_type
row_width = str(anndata_m.row_width()[0][0])
column_width = str(anndata_m.column_width()[0][0])
results_dict["row width"] = row_width
results_dict["column width"] = column_width



Method data_type took 0.0000 seconds
Method get_int_size took 0.0000 seconds
Method row_width took 0.0000 seconds
Method get_int_size took 0.0000 seconds
Method column_width took 0.0000 seconds


In [11]:
scdl_path = "memmap_93bc4573"
scdl_m = SCDLMetrics(memmap_dir=scdl_path, adatapath=anndatapath, row_width = row_width, column_width = column_width, data_type = data_type)
results_dict["SCDL Dataset from AnnData Time (s)"] = scdl_m.create_from_adata()[1]
results_dict["SCDL Dataset save time (s)"] = scdl_m.save()[1]

Method create_from_adata took 1.8452 seconds
Method save took 0.0277 seconds


In [12]:
results_dict["SCDL Dataset Load Time (s)"] = scdl_m.load_backed()[1]
results_dict["SCDL Dataset Max Calculation Time (s)"] = scdl_m.max()[1]
results_dict["SCDL Dataset Min Calculation Time (s)"] = scdl_m.min()[1]
results_dict["SCDL Dataset Mean Calculation Time (s)"] = scdl_m.mean()[1]
results_dict["Dataset Sparsity"] = scdl_m.sparsity_stats()[0]

results_dict["SCDL Dataset Size on Disk (MB)"] = scdl_m.size_disk_bytes()[0]/(1_024**2)
results_dict["SCDL Dataset Size in Memory (MB)"] = scdl_m.size_mem_dataset_bytes()[0]/(1_024**2)


results_dict["SCDL Time to retrieve a random batch of 100 cells backed on disk (s)"] =  scdl_m.random_rows()[1]
results_dict["SCDL Time to retrive 100 values (s)"]  = scdl_m.random_values(random_samples = 100)
results_dict["SCDL Time to iterate over Dataset (s)"] = scdl_m.iterate_dl()[1]

Method load_backed took 0.0425 seconds
Method max took 0.0090 seconds
Method min took 0.0047 seconds
Method mean took 0.0055 seconds
Method sparsity_stats took 0.0000 seconds
Method size_disk_bytes took 0.0019 seconds
Method size_mem_dataset_bytes took 0.0000 seconds
Method random_rows took 0.1953 seconds


/home/pbinder/bionemo-fw-ea/sub-packages/bionemo-scdl/src/bionemo/scdl/util/torch_dataloader_utils.py:39: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  batch_sparse_tensor = torch.sparse_csr_tensor(batch_rows, batch_cols, batch_values, size=(len(batch), max_pointer))


Method iterate_dl took 0.6436 seconds


In [13]:
results_dict

{'anndata file': '97e96fb1-8caf-4f08-9174-27308eabd4ea.h5ad',
 'AnnData Dataset Backed Load Time (s)': 0.21448874473571777,
 'AnnData Dataset Load Time (s)': 1.4693443775177002,
 'AnnData Dataset Max Calculation Time (s)': 0.0050313472747802734,
 'AnnData Dataset Min Calculation Time (s)': 0.004914045333862305,
 'AnnData Dataset Mean Calculation Time (s)': 0.005697011947631836,
 'AnnData Dataset Size on Disk (MB)': 144.6051368713379,
 'AnnData Dataset Size in Memory (MB)': 235.33053874969482,
 'AnnData Backed Dataset Size in Memory (MB)': 29.642348289489746,
 'AnnData Time to retrieve a random batch of 100 cells loaded in memory (s)': 0.014048576354980469,
 'AnnData Time to retrieve a random batch of 100 cells backed on disk (s)': 0.012162923812866211,
 'data type': 'float32',
 'row width': 'uint32',
 'column width': 'uint16',
 'SCDL Dataset from AnnData Time (s)': 1.8451993465423584,
 'SCDL Dataset save time (s)': 0.027729034423828125,
 'SCDL Dataset Load Time (s)': 0.0425353050231933

In [14]:
dicts = [results_dict]
combined = {key: [d[key] for d in dicts] for key in dicts[0]}


In [15]:
df = pd.DataFrame(combined)

In [18]:
d = pd.read_csv("results.csv")

In [19]:
d

,Unnamed: 0,anndata file,AnnData Dataset Backed Load Time (s),AnnData Dataset Load Time (s),AnnData Dataset Max Calculation Time (s),AnnData Dataset Min Calculation Time (s),AnnData Dataset Mean Calculation Time (s),AnnData Dataset Size on Disk (MB),AnnData Dataset Size in Memory (MB),AnnData Backed Dataset Size in Memory (MB),...,SCDL Dataset Load Time (s),SCDL Dataset Max Calculation Time (s),SCDL Dataset Min Calculation Time (s),SCDL Dataset Mean Calculation Time (s),Dataset Sparsity,SCDL Dataset Size on Disk (MB),SCDL Dataset Size in Memory (MB),SCDL Time to retrieve a random batch of 1000 cells backed on disk (s),SCDL Time to retrive 1000 values (s),SCDL Time to iterate over Dataset (s)
0,0,5d6308fd-76e2-45b1-b7a1-1f671e2097b7,0.400354,6.339357,0.021927,0.021588,0.023746,913.051188,965.713401,52.495967,...,0.061664,0.036766,0.020856,0.023929,0.942648,916.956355,0.000046,0.033606,0.66708,1.072148
